In [1]:
import pandas as pd
import numpy as np
import os
import glob

pd.options.display.max_columns = None
pd.options.display.max_rows = None

os.chdir('/slurm_storage/mbopf/projects/MFM_bopf/src/multi_predict/MLP/output')
#os.chdir('/MFMDatasets/MFM_bopf/src/multi_predict/NB/output')
results_df = pd.DataFrame()
#files = glob.glob("*-NB-*.csv")
files = glob.glob("*-MLP-*.csv")
#files = glob.glob("transfus_yes*-MLP-*.csv")
#files = glob.glob("transfus_yes*-MLP-*]_0.*_500*.csv")
#files = glob.glob("transfus_hyster*-MLP-*.csv")
#files = glob.glob("transfus_yes*-MLP-*.csv")
#files = glob.glob("transfus_yes*-MLP-*.csv")

In [2]:
files = sorted(files, reverse=True)
for i in range(len(files)-1):
    if files[i][:-20] == files[i+1][:-20]:
        print(f'{files[i]}')

transfus_yes-1-RAND-MLP-relu_1e-05_[64,32,16,8,4]_0.001_500_7_adam-1-1.0-20201023_161012.csv
transfus_yes-1-RAND-MLP-relu_1e-05_[52,26,13,6]_0.001_500_7_adam-1-1.0-20201023_161021.csv


In [3]:
#files.sort(key=os.path.getmtime) # Why???
prev_f = ""
for idx, f in enumerate(files):
    if prev_f[:-20] == f[:-20]:   #if the filenames only differ by timestamp
        print(f'{prev_f}')
        continue   #skip "duplicate"
        
    if results_df.empty:
        results_df = pd.read_csv(f, header=None, index_col=0, names=[idx])
    else:
        this_df = pd.read_csv(f, header=None, index_col=0, names=[idx])
#        if 'p_learning_rate_init' in this_df.index:
#            print(f'p_learning_rate_init = {this_df.loc["p_learning_rate_init"]}')
        results_df = results_df.merge(this_df, left_index=True, right_index=True, how="right")
        #results_df = results_df.merge(this_df, left_index=True, right_index=True, how="left")
    prev_f = f
results_df

transfus_yes-1-RAND-MLP-relu_1e-05_[64,32,16,8,4]_0.001_500_7_adam-1-1.0-20201023_161012.csv
transfus_yes-1-RAND-MLP-relu_1e-05_[52,26,13,6]_0.001_500_7_adam-1-1.0-20201023_161021.csv


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,46,47,48,49,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272
CLF_time(min),0.083,0.086,0.082,0.090,0.132,0.082,0.078,0.056,0.088,0.082,0.005,0.198,0.094,0.169,0.098,0.115,0.030,0.032,0.082,0.111,0.075,0.034,0.069,0.093,0.001,0.142,0.110,0.061,0.102,0.044,0.062,0.142,0.080,0.034,0.077,0.075,0.042,0.089,0.052,0.007,0.167,0.119,0.015,0.019,0.140,0.502,0.082,0.001,0.170,0.115,0.015,0.110,0.153,0.320,0.055,0.428,0.136,0.379,0.181,0.083,0.115,0.233,0.194,0.172,0.131,0.103,0.115,0.123,0.092,0.319,0.156,0.107,0.121,0.139,0.090,0.208,0.105,0.085,0.105,0.100,0.052,0.149,0.296,0.338,0.100,0.994,0.096,0.085,0.086,0.164,0.147,0.100,0.546,0.128,0.112,0.086,0.303,0.154,0.126,0.846,0.117,0.190,0.095,0.182,0.087,0.033,0.161,0.171,0.184,0.540,0.147,0.055,0.066,0.175,0.106,0.107,0.078,0.038,0.078,0.039,0.003,0.133,0.166,0.032,0.023,0.125,0.510,0.084,0.001,0.215,0.087,0.033,0.075,0.217,0.320,0.054,0.078,0.041,0.083,0.110,0.038,0.045,0.035,0.110,0.112,0.123,0.070,0.164,0.094,0.162,0.101,0.099,0.110,0.028,0.128,0.115,0.091,0.076,0.125,0.217,0.052,0.076,0.075,0.049,0.181,0.058,0.250,0.130,0.189,0.210,0.094,0.101,0.094,0.110,0.237,0.139,0.091,0.103,0.101,0.087,0.159,0.120,0.237,0.117,0.173,0.132,0.059,0.093,0.118,0.122,0.068,0.222,0.094,0.990,0.152,0.046,0.207,0.180,0.911,0.398,0.229,0.087,0.191,0.070,0.235,0.243,1.200,0.149,0.175,1.123,0.200,0.198,0.191,0.189,0.152,0.195,0.069,0.245,0.131,0.088,0.462,0.071,0.247,0.557,0.071,1.098,0.128,0.175,0.160,0.174,1.312,0.016,0.012,0.023,0.014,0.012,0.013,0.011,0.027,0.016,0.012,0.023,0.015,0.012,0.014,0.011,0.018,0.016,0.012,0.021,0.014,0.013,0.014,0.012,0.017,0.014,0.014,0.024,0.013,0.011,0.014,0.012,0.022,0.017,0.013,0.024,0.015,0.013,0.015,0.013,0.017
target,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_yes,transfus_ye

In [4]:
results_T = results_df.T
results_T.sort_values(by=['ROC_AUC'], ascending=False, inplace=True)
small = results_T[['target','under_alg','p_activation','p_alpha','p_hidden_layer_sizes','p_random_state','p_solver','ROC_AUC','MCC']]
#small['p_alpha'] = pd.to_numeric(small['p_alpha'])
#small.sort_values(by=['p_alpha'], inplace=True)
#small.sort_values(by=['under_alg','p_alpha'], ascending=False, inplace=True)
small

,target,under_alg,p_activation,p_alpha,p_hidden_layer_sizes,p_random_state,p_solver,ROC_AUC,MCC
131,transfus_yes,RAND,relu,0.001,"[52, 26, 13, 6]",7,adam,0.8108,0.6288
135,transfus_yes,RAND,relu,0.001,"[52, 26, 13, 6]",7,adam,0.8108,0.6288
41,transfus_yes,RAND,relu,1e-05,"[64, 32, 16, 8, 4]",13,adam,0.8101,0.6274
80,transfus_yes,RAND,relu,0.3,"[52, 26, 13, 6]",11,adam,0.8098,0.6233
112,transfus_yes,RAND,relu,0.05,"[64, 32, 16, 8, 4]",7,adam,0.8095,0.6242
44,transfus_yes,RAND,relu,1e-05,"[64, 32, 16, 8, 4]",7,adam,0.8092,0.6250
61,transfus_yes,RAND,relu,1.5,"[52, 26, 13, 6]",13,adam,0.8092,0.6221
40,transfus_yes,RAND,relu,1e-05,"[64, 32, 16, 8, 4]",7,adam,0.8092,0.6250
161,transfus_hyster,high_Age,relu,1e-05,"[64, 32, 16, 8, 4]",7,adam,0.8089,0.6258
52,transfus_yes,RAND,relu,1e-05,"[52, 26, 13, 6]",7,adam,0.8088,0.6195


In [5]:
os.chdir('/slurm_storage/mbopf/projects/MFM_bopf/src/multi_predict/MLP/output')
#files = glob.glob("*-NB-*.out")
#files = glob.glob("*-LR-*.out")
#files = glob.glob("*-RF-*.out")
#files = glob.glob("*-SVC-*.out")
#files = glob.glob("*-LSVC-*.out")
#files = glob.glob("*-MLP-*.out")

#files = glob.glob("*high_Age-MLP-*.out")
#ROC_CUTOFF = 0.795
#ROC_CUTOFF = 0.80
ROC_CUTOFF = 0.805
#ROC_CUTOFF = 0.0
#files = glob.glob("transfus_yes*-MLP-*.out")
files = glob.glob("*-MLP-*.out")

In [6]:
files.sort()
for i in range(len(files)-1):
    if files[i][:-20] == files[i+1][:-20]:
        print(f'{files[i]}')

transfus_yes-1-RAND-MLP-relu_1e-05_[52,26,13,6]_0.001_500_7_adam-1-1.0-20201023_140150.out
transfus_yes-1-RAND-MLP-relu_1e-05_[64,32,16,8,4]_0.001_500_7_adam-1-1.0-20201023_140141.out


In [7]:
from tokenize import tokenize

pd.options.display.float_format = '{:,.4f}'.format
files.sort()
prev_f = ""
coeffs_df = pd.DataFrame()
coeffs_rank_df = pd.DataFrame()
for idx, f in enumerate(files):
    if prev_f[:-20] == f[:-20]:   #if the filenames only differ by timestamp
        print(f'{prev_f}')
        continue   #skip "duplicate"
 
    with open(f, 'r') as file:
        found_coef = False
        cutoff = False
        coef_series = pd.Series()
        ind_vars = []
        coef_vals = []
        for line in file:
            # Add ROC_AUC cutoff
            if "ROC_AUC" in line:
                roc_auc = line.split()[2]   
                if float(roc_auc) < ROC_CUTOFF:
                    #print(f'roc_auc = {roc_auc}')
                    cutoff = True
                    break    # Stop processing this file; poor performance
            
            if "coeffs =" in line:
                found_coef = True
                continue
            
            if found_coef:
                var_coef = line.split()
                if var_coef[0] != 'dtype:':
                    ind_vars.append(var_coef[0])
                    coef_vals.append(float(var_coef[1]))
                
        if not cutoff:
            if coeffs_df.empty:
                coeffs_df = pd.DataFrame(data=coef_vals, index=ind_vars, columns=[idx])
                coeffs_rank_df = pd.DataFrame(data=coeffs_df[idx].rank(ascending=False).astype(int), columns=[idx])
            else:
                this_df = pd.DataFrame(data=coef_vals, index=ind_vars, columns=[idx])
                this_rank_df = pd.DataFrame(data=this_df[idx].rank(ascending=False).astype(int), columns=[idx])
                coeffs_df = coeffs_df.merge(this_df, left_index=True, right_index=True, how="right")
                coeffs_rank_df = coeffs_rank_df.merge(this_rank_df, left_index=True, right_index=True, how="right")

    prev_f = f
    
#if 'high_Age' in coeffs_df.index:
#    coeffs_df.drop('high_Age', inplace=True)
#    coeffs_rank_df.drop('high_Age', inplace=True)
#if 'Insurance' in coeffs_df.index:
#    coeffs_df.drop('Insurance', inplace=True)
#    coeffs_rank_df.drop('Insurance', inplace=True)
coeffs_df

transfus_yes-1-RAND-MLP-relu_1e-05_[52,26,13,6]_0.001_500_7_adam-1-1.0-20201023_140150.out
transfus_yes-1-RAND-MLP-relu_1e-05_[64,32,16,8,4]_0.001_500_7_adam-1-1.0-20201023_140141.out


,111,137,141,148,154,155,156,159,160,162,167,169,182,192,202,211,212,217,218,220,223,227,231,232,235
abruptio9,0.9897,0.5929,0.5929,0.6538,0.2594,0.1118,0.0709,0.1017,0.0766,0.0425,0.0434,0.0425,0.0369,0.0277,0.0135,0.0062,0.0062,0.2464,0.7141,2.1846,0.7141,0.7767,0.6453,0.7767,0.6133
Delmode,0.3592,0.3880,0.3880,0.3189,0.2599,0.2341,0.2208,0.2277,0.2783,0.2032,0.2006,0.2032,0.1688,0.1397,0.1146,0.0764,0.0529,0.1602,0.3859,0.8758,0.3859,0.3197,0.3176,0.3197,0.2862
HosEpiNurse,0.3330,0.3733,0.3733,0.2376,0.1485,0.1428,0.1289,0.0904,0.1549,0.1201,0.0995,0.1201,0.0670,0.0614,0.0287,0.0114,0.0118,0.1287,0.3444,0.7451,0.3444,0.2303,0.2021,0.2303,0.2279
Intratocolytix,0.2616,0.3060,0.3060,0.2753,0.2196,0.2104,0.1855,0.2038,0.2149,0.1995,0.1654,0.1995,0.1667,0.1536,0.1299,0.1052,0.0571,0.1616,0.3024,0.5931,0.3024,0.2424,0.2705,0.2424,0.2273
Anteanemia,0.5526,0.4715,0.4715,0.3577,0.1789,0.1672,0.1339,0.1144,0.1301,0.1265,0.1118,0.1265,0.0753,0.0690,0.0392,0.0144,0.0158,0.1271,0.3921,0.9654,0.3921,0.2776,0.2690,0.2776,0.2198
Operative,0.5139,0.3140,0.3140,0.2567,0.1196,0.0754,0.0535,0.0615,0.0481,0.0477,0.0409,0.0477,0.0308,0.0264,0.0103,0.0032,0.0019,0.1286,0.2701,0.5635,0.2701,0.2384,0.2063,0.2384,0.2037
Marital,0.2244,0.2672,0.2672,0.2276,0.1359,0.1216,0.1086,0.0903,0.1230,0.0967,0.0809,0.0967,0.0532,0.0641,0.0337,0.0197,0.0166,0.1227,0.2296,0.6347,0.2296,0.2176,0.1802,0.2176,0.2032
Anteprevia,0.2268,0.2244,0.2244,0.1995,0.1352,0.1103,0.0806,0.0785,0.0870,0.0769,0.0692,0.0769,0.0545,0.0410,0.0209,0.0086,0.0098,0.1114,0.1922,0.5962,0.1922,0.1931,0.1722,0.1931,0.1805
Meconium,0.1760,0.2078,0.2078,0.1500,0.1153,0.1019,0.0630,0.0741,0.0328,0.0433,0.0417,0.0433,0.0457,0.0264,0.0116,0.0054,0.0053,0.1193,0.1604,0.2885,0.1604,0.1288,0.1308,0.1288,0.1716
abruption,0.2310,0.2122,0.2122,0.1964,0.1469,0.1059,0.0991,0.0839,0.0888,0.0849,0.0819,0.0849,0.0552,0.0445,0.0256,0.0110,0.0110,0.1190,0.1969,0.5733,0.1969,0.1790,0.1781,0.1790,0.1673


In [8]:
coeffs_rank_df['mean_rank'] = coeffs_rank_df.apply(np.mean, axis=1).rank().astype(int)
coeffs_rank_df = coeffs_rank_df.astype('Int8')
coeffs_rank_df.sort_values('mean_rank', inplace=True)
coeffs_rank_df

,111,137,141,148,154,155,156,159,160,162,167,169,182,192,202,211,212,217,218,220,223,227,231,232,235,mean_rank
Delmode,5,3,3,3,1,1,1,1,1,1,1,1,1,2,2,2,2,3,3,3,3,2,2,2,2,1
Intratocolytix,11,7,7,4,3,2,2,2,2,2,2,2,2,1,1,1,1,2,5,12,5,4,3,4,4,2
Anteanemia,2,2,2,2,4,3,4,5,5,3,4,3,19,13,16,23,18,9,2,2,2,3,4,3,5,3
HosEpiNurse,7,4,4,6,5,4,5,25,3,4,11,4,24,19,24,25,22,6,4,4,4,6,6,6,3,4
Marital,15,9,9,8,13,7,12,26,6,9,21,9,29,14,20,19,17,13,8,8,8,8,11,8,7,5
Education,21,21,21,23,19,16,15,3,4,6,24,6,12,12,12,5,9,12,24,21,24,16,17,16,26,6
ROMmeth,17,12,12,22,15,6,7,14,8,7,5,7,22,22,19,16,15,42,15,18,15,18,22,18,27,7
abruptio9,1,1,1,1,2,10,37,17,32,43,41,43,41,40,42,41,41,1,1,1,1,1,1,1,1,8
Dilat_lst,20,24,24,10,8,5,17,7,28,12,16,12,15,20,21,30,26,23,19,22,19,21,10,21,23,9
IntraMgSO,8,11,11,9,7,31,26,22,33,22,28,22,35,35,31,27,23,10,11,9,11,10,7,10,16,10


In [9]:
coeffs_rank_inv = coeffs_rank_df.T
combo = pd.concat([small, coeffs_rank_inv['abruptio9']], axis=1, join='inner')
combo

,target,under_alg,p_activation,p_alpha,p_hidden_layer_sizes,p_random_state,p_solver,ROC_AUC,MCC,abruptio9
111,transfus_yes,RAND,relu,0.1,"[52, 26, 13, 6]",7,adam,0.8021,0.6101,1
137,transfus_yes,RAND,relu,0.001,"[200, 100, 50]",7,adam,0.7982,0.6027,1
167,transfus_hyster,high_Age,relu,0.001,"[52, 26, 13, 6]",7,adam,0.7933,0.5873,41
162,transfus_hyster,high_Age,relu,1e-05,"[52, 26, 13, 6]",7,adam,0.7911,0.5825,43
141,transfus_yes,Insurance,relu,1e-05,"[64, 32, 16, 8, 4]",7,adam,0.7850,0.5764,1
156,transfus_hyster,momrace_new,relu,0.1,"[64, 32, 16, 8, 4]",7,adam,0.7739,0.5667,37
182,transfus_hyster,Insurance,relu,0.001,"[64, 32, 16, 8, 4]",7,adam,0.7723,0.5668,41
154,transfus_hyster,momrace_new,relu,1e-05,"[52, 26, 13, 6]",7,adam,0.7720,0.5601,2
159,transfus_hyster,momrace_new,relu,0.001,"[52, 26, 13, 6]",7,adam,0.7599,0.5495,17
160,transfus_hyster,momrace_new,relu,0.001,"[200, 100, 50]",7,adam,0.7589,0.5428,32


In [10]:
coeffs_df_inv = coeffs_df.T
combo2 = pd.concat([combo, coeffs_df_inv['abruptio9']], axis=1, join='inner')
combo2

,target,under_alg,p_activation,p_alpha,p_hidden_layer_sizes,p_random_state,p_solver,ROC_AUC,MCC,abruptio9,abruptio9
111,transfus_yes,RAND,relu,0.1,"[52, 26, 13, 6]",7,adam,0.8021,0.6101,1,0.9897
137,transfus_yes,RAND,relu,0.001,"[200, 100, 50]",7,adam,0.7982,0.6027,1,0.5929
167,transfus_hyster,high_Age,relu,0.001,"[52, 26, 13, 6]",7,adam,0.7933,0.5873,41,0.0434
162,transfus_hyster,high_Age,relu,1e-05,"[52, 26, 13, 6]",7,adam,0.7911,0.5825,43,0.0425
141,transfus_yes,Insurance,relu,1e-05,"[64, 32, 16, 8, 4]",7,adam,0.7850,0.5764,1,0.5929
156,transfus_hyster,momrace_new,relu,0.1,"[64, 32, 16, 8, 4]",7,adam,0.7739,0.5667,37,0.0709
182,transfus_hyster,Insurance,relu,0.001,"[64, 32, 16, 8, 4]",7,adam,0.7723,0.5668,41,0.0369
154,transfus_hyster,momrace_new,relu,1e-05,"[52, 26, 13, 6]",7,adam,0.7720,0.5601,2,0.2594
159,transfus_hyster,momrace_new,relu,0.001,"[52, 26, 13, 6]",7,adam,0.7599,0.5495,17,0.1017
160,transfus_hyster,momrace_new,relu,0.001,"[200, 100, 50]",7,adam,0.7589,0.5428,32,0.0766


In [11]:
abruptio9 = coeffs_rank_df.loc['abruptio9']
abruptio9.drop('mean_rank')

111     1
137     1
141     1
148     1
154     2
155    10
156    37
159    17
160    32
162    43
167    41
169    43
182    41
192    40
202    42
211    41
212    41
217     1
218     1
220     1
223     1
227     1
231     1
232     1
235     1
Name: abruptio9, dtype: Int8